# Running a CAT based on a synthetic correct/incorrect MIRT model

This example shows how to run a CAT based on a synthetic correct/incorrect
MIRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: DRuleItemCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
dims = 2
using ComputerAdaptiveTesting.DummyData: dummy_full, std_mv_normal, SimpleItemBankSpec, StdModel4PL
using ComputerAdaptiveTesting.MathTraits
using ComputerAdaptiveTesting.Responses: BooleanResponse

TODO: pass in dims

In [3]:
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel4PL(), VectorContinuousDomain(), BooleanResponse()),
    dims;
    num_questions=10,
    num_testees=2
)

(ComputerAdaptiveTesting.ItemBanks.SlipItemBank{ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}}([0.3167605040071, 0.058024485041318186, 0.17080972424646582, 0.0, 0.0650106395434497, 0.024514067439122152, 0.3676831354191526, 0.21200051019088334, 0.25044009645043114, 0.08973408348834873], ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}([0.1933944990651114, 0.0, 0.14893485761357622, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2678059047730411, 0.0], ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [0.06929282203884661, 1.0607818491823704, 0.5855069355856899, 0.4149245743595446, 0.6523325148580902, -0.20571112263173627, -0.30601201837833636, -0.9538542100726435, 2.2745060890296194, 0.31797678763891774], [0.7746746218255881 0.5745843852735562 … 1.

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [4]:
max_questions = 9
integrator = CubaIntegrator([-6.0, -6.0], [6.0, 6.0], CubaVegas(); rtol=1e-2)
ability_estimator = MeanAbilityEstimator(PriorAbilityEstimator(std_mv_normal(dims)), integrator)
rules = CatRules(
    ability_estimator,
    DRuleItemCriterion(ability_estimator),
    FixedItemsTerminationCondition(max_questions)
)

CatRules
  next_item: ComputerAdaptiveTesting.NextItemRules.ItemStrategyNextItemRule{ComputerAdaptiveTesting.NextItemRules.ExhaustiveSearch1Ply, ComputerAdaptiveTesting.NextItemRules.DRuleItemCriterion{ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integrators.CubaIntegrator{ComputerAdaptiveTesting.Integrators.CubaVegas, Base.Pairs{Symbol, Float64, Tuple{Symbol}, NamedTuple{(:rtol,), Tuple{Float64}}}}}}}}
  termination_condition: ComputerAdaptiveTesting.TerminationConditions.FixedItemsTerminationCondition
  ability_estimator: ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integra

XXX: We shouldn't need to specify xs here since the distributions are not used -- rework

In [5]:
points = 3
xs = repeat(range(-2.5, 2.5, length=points)', dims, 1)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator, abilities)
for testee_idx in axes(responses, 2)
    @debug "Running for testee" testee_idx
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[:, testee_idx]
    abs_err = sum(abs.(θ .- true_θ))
    @info "convergence" true_θ θ abs_err
end

┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     -0.14759594784258503
│     -2.426981867258469
│   θ =
│    2-element Vector{Float64}:
│     -1.043393322419399
│     -0.5508576832093959
└   abs_err = 2.771921558625887
┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     1.185943022380581
│     0.03438749342775359
│   θ =
│    2-element Vector{Float64}:
│     1.2290131156432293
│     0.717429222089576
└   abs_err = 0.7261118219244709


Make a plot showing how the estimated value converges during the CAT.

In [6]:
conv_lines_fig = ability_convergence_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*